In [53]:
import numpy as np
import pandas as pd
from scipy.stats import binom
import matplotlib
import matplotlib.pyplot as plt   
import metakernel
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets    
from ipywidgets import interact, interactive, fixed, interact_manual

In [54]:
def find_two_largest_indices(list_of_lists, j):
    # Convert the column at index j to a numpy array
    column = np.array([row[j] for row in list_of_lists])

    # Find the indices of the two largest values
    indices = np.argpartition(column, -2)[-2:]

    # Sort the indices so that the first index corresponds to the largest value
    # and the second index corresponds to the second largest value
    indices = indices[np.argsort(column[indices])][::-1]

    return indices.tolist()

In [55]:
# size_sample_space, size_parameter_space are positive integers
# prior is list of length size_parameter_space of non-negative numbers that sum to 1
# likelihood is a list of length size_parameter_space 
# where each element is a list of length size_sample_space numbers that sum to 1

def bayes(size_sample_space,size_parameter_space,prior,likelihood, xlim0, xlim1, observed_value = None, ι = None):


    sample_space = list(range(size_sample_space))

    parameter_space = list(range(size_parameter_space))


    max_value = max(max(sublist) for sublist in likelihood)


    evidence =  []

    for j in sample_space:
        evidence.append(sum([prior[i]*likelihood[i][j] for i in parameter_space]))



    posterior = [[None]*size_sample_space for _ in range(size_parameter_space)]

    posteriorupvalue= [[None]*size_parameter_space for _ in range(size_sample_space)]

    for i in parameter_space:
        for j in sample_space:
            posterior[i][j] = ((likelihood[i][j]*prior[i]) / (evidence[j]+ 1e-10))

    for j in sample_space:
        for i in parameter_space:
            posteriorupvalue[j][i] = sum([posterior[i1][j] for i1 in parameter_space if i1 > i])

    fig, axs = plt.subplots(1, 3, figsize=(10, 5))  # 1 row, 3 columns



    for i in parameter_space:
        line, = axs[0].plot(sample_space, likelihood[i])
    axs[0].plot(sample_space, evidence, '--', label='evidence', color='purple')        
    axs[0].set_ylim(0, max_value+max_value/10)
    axs[0].set_xlim(0, xlim0)    
    axs[0].set_title('likelihood p(x | $\\theta$)')
    axs[0].legend()
    axs[0].legend(loc='upper right')
    axs[0].set_xticks(np.round(np.linspace(0, xlim1, num=11), 0))
    axs[0].tick_params(axis='x', labelsize=5)          

    for i in parameter_space:
        axs[1].plot(sample_space, posterior[i]) 
    if observed_value != None:
        axs[1].axvline(x=observed_value, color='black', linestyle='--', label='x')            
        indices = find_two_largest_indices(posterior, observed_value)
        axs[1].axhline(y=posterior[indices[0]][observed_value], color='#ffd700', linestyle='--', label='1st')  
        axs[1].axhline(y=posterior[indices[1]][observed_value], color='#C0C0C0', linestyle='--', label='2nd')  
    axs[1].legend()
    axs[1].legend(loc='upper right')

    axs[1].set_ylim(0, 1.1)
    axs[1].set_title('posterior p($\\theta$  | x)')
    axs[1].set_xlim(0, xlim1)    
    axs[1].set_yticks([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1])
    axs[1].set_xticks(np.round(np.linspace(0, xlim1, num=11), 0))  
    axs[1].tick_params(axis='x', labelsize=5)    

    if ι != None:
        axs[2].axvline(x=ι, color='g', linestyle='--', label='ι') 
        axs[2].axhline(y=posteriorupvalue[observed_value][ι], color='red', linestyle='--')               
    axs[2].plot(parameter_space, posteriorupvalue[observed_value]) 
    axs[2].set_title('Pr($\\theta> \\iota$  | x), $\\iota$ is Binom(%i, %1.2f)' % (size_sample_space-1, (ι+1)/(size_parameter_space+1))) 
    axs[2].set_yticks([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1])    
    axs[2].legend()
    axs[2].set_ylim(0, 1.1)    
    axs[2].legend(loc='center right')    

    return posterior



In [56]:
def bayes_binomials(n,m, prior, observed_value = None, ι=None):


    my_sample_space = np.arange(0, n+1)

    my_parameter_space = np.arange(0, m)

    my_likelihood = [[None]*m for _ in range(m)]

    for i in range(m):

        my_likelihood[i] = binom.pmf(my_sample_space, n, (i+1)/(m+1))

    bayes(n+1,m,prior,my_likelihood, n, n, observed_value, ι)


In [57]:
def bayes_binomials_uniform_prior(n,m, x = None, ι = None):

    

    prior = [1/(m+1)]*(m)

    bayes_binomials(n,m,prior, x, ι)

In [58]:
def update_x_range(*args):
    x_slider.max = n_slider.value

def update_ι_range(*args):
    ι_slider.max = m_slider.value - 1

In [59]:
n_slider = widgets.IntSlider(min=1, max=5000, step=1, value=100)
m_slider = widgets.IntSlider(min=1, max=99, step=1, value=9)
x_slider = widgets.IntSlider(min=1, max=n_slider.value, step=1, value=40)
ι_slider = widgets.IntSlider(min=1, max=m_slider.value - 1, step=0, value=3)

n_slider.observe(update_x_range, 'value')
m_slider.observe(update_ι_range, 'value')


interact(bayes_binomials_uniform_prior, n=n_slider, m=m_slider, x=x_slider, ι=ι_slider)


interactive(children=(IntSlider(value=100, description='n', max=5000, min=1), IntSlider(value=9, description='…

<function __main__.bayes_binomials_uniform_prior(n, m, x=None, ι=None)>